## Chat Model Messages: An Overview

This document details how messages are structured and used within chat models, particularly focusing on how LangChain handles them.

### Core Concepts

*   **Messages** are the basic units of communication in chat models, representing both inputs and outputs.
*   Each message has a **`role`** and **`content`**, along with optional metadata.
*   **LangChain** provides a consistent way to manage messages across different chat models.

### Message Components

*   **`Role`**:  This categorises the message and helps the chat model respond appropriately. Key roles include:
    *   **`system`**: Used to instruct the model and provide context.
    *   **`user`**: Represents input from a user.
    *   **`assistant`**: Represents a response from the model.
    *   **`tool`**: Passes results of a tool invocation back to the model.
    *   `function`: A legacy role for OpenAI's function-calling API, `tool` should be used instead.
*   **`Content`**: The actual message information, can be text or multimodal data (images, audio, video). Most models primarily use text.
*   **Other Data**: Messages may include:
    *   **`ID`**: A unique identifier.
    *   **`Name`**: Differentiates entities with the same role.
    *   **`Metadata`**: Additional info, like timestamps or token usage.
    *   **`Tool Calls`**: Requests from the model to use tools.

### Conversation Structure

*   A typical conversation consists of a structured sequence of messages, usually alternating between **`user`** and **`assistant`** messages.

### LangChain Message Types

*   LangChain represents messages as Python objects inheriting from `BaseMessage`.
*   Key message types:
    *   **`SystemMessage`**: Corresponds to the `system` role.
    *   **`HumanMessage`**: Corresponds to the `user` role.
    *   **`AIMessage`**: Corresponds to the `assistant` role.
    *   **`AIMessageChunk`**: Used for streaming responses, also `assistant` role.
    *   **`ToolMessage`**: Corresponds to the `tool` role.
*   Other message types:
    *   **`RemoveMessage`**: Manages chat history, not associated with any role.
    *   **`FunctionMessage`**: Legacy message for the `function` role, use `ToolMessage` instead.

### Specific Message Details

*   **`SystemMessage`**:
    *   Primes the AI model by providing context.
    *   Implemented differently by providers (message role, API parameter, or no support).
    *   LangChain adapts based on provider capabilities and tries to incorporate the content in a HumanMessage if system messages are not supported by the provider.
*   **`HumanMessage`**:
    *   Represents user input, usually text, sometimes multimodal content.
    *   LangChain can convert a string input into a `HumanMessage`.
*   **`AIMessage`**:
    *   Model's response, may include text or tool calls.
     *  The `content` property is not standardized and can be text or a list of dictionaries.
    *   Includes attributes like `content`, `tool_calls`, `invalid_tool_calls`, `usage_metadata`, `id`, and `response_metadata`.
*   **`AIMessageChunk`**:
    *   For streaming responses.
    *   Can be merged into a single `AIMessage` using the `+` operator.
*   **`ToolMessage`**:
    *   Contains the result of tool calls, with a `tool_call_id` and an `artifact` field.
*   **`RemoveMessage`**: For managing chat history in LangGraph.
*   **`(Legacy) FunctionMessage`**: For the legacy function-calling API. `ToolMessage` should be used instead.

### OpenAI Format

*   Chat models accept OpenAI's format as input, a dictionary with `role` and `content`.
*   The output of models will be in terms of LangChain messages which can be converted to OpenAI format using `convert_to_openai_messages`.


In [1]:
from langchain_core.messages import HumanMessage
from langchain_ollama.chat_models import ChatOllama

model = ChatOllama(model='llama3.1')

model.invoke([HumanMessage(content="Hello, how are you?")])

AIMessage(content="I'm just a computer program, so I don't have feelings or emotions like humans do. But thank you for asking! How can I assist you today?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-12-24T19:24:47.002150976Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1612224118, 'load_duration': 1316112365, 'prompt_eval_count': 16, 'prompt_eval_duration': 52000000, 'eval_count': 33, 'eval_duration': 243000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1db95c60-3523-4083-8bb0-1ef520f1b79c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 33, 'total_tokens': 49})

In [2]:
model.invoke("Hello, how are you?")

AIMessage(content="I'm just a computer program, so I don't have feelings or emotions like humans do. But thank you for asking! How can I assist you today? Do you have any questions or topics you'd like to discuss?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-12-24T19:24:47.365290728Z', 'done': True, 'done_reason': 'stop', 'total_duration': 357220106, 'load_duration': 12503855, 'prompt_eval_count': 16, 'prompt_eval_duration': 1000000, 'eval_count': 46, 'eval_duration': 342000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-2077083b-92b4-4eb8-a8f1-1124b2ada253-0', usage_metadata={'input_tokens': 16, 'output_tokens': 46, 'total_tokens': 62})

In [3]:
messages = None
for index, chunk in enumerate(model.stream([HumanMessage("what color is the sky?")])):
    if messages is None:
        messages = chunk
    else:
        messages += chunk
    if index > 5:
        continue
    print(chunk)

print(messages)

content='The' additional_kwargs={} response_metadata={} id='run-0994f30a-0747-45d9-8fcb-52b7444c6a70'
content=' answer' additional_kwargs={} response_metadata={} id='run-0994f30a-0747-45d9-8fcb-52b7444c6a70'
content=',' additional_kwargs={} response_metadata={} id='run-0994f30a-0747-45d9-8fcb-52b7444c6a70'
content=' of' additional_kwargs={} response_metadata={} id='run-0994f30a-0747-45d9-8fcb-52b7444c6a70'
content=' course' additional_kwargs={} response_metadata={} id='run-0994f30a-0747-45d9-8fcb-52b7444c6a70'
content=',' additional_kwargs={} response_metadata={} id='run-0994f30a-0747-45d9-8fcb-52b7444c6a70'
content="The answer, of course, depends on the time of day and atmospheric conditions.\n\n**During the Day:**\nWhen the sun is overhead, the sky appears to be **blue**. This is because when sunlight enters Earth's atmosphere, it scatters in all directions due to interactions with tiny molecules of gases such as nitrogen and oxygen. The shorter (blue) wavelengths are scattered more 

In [15]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

class SecurityProjectManager(HumanMessage):
    def pretty_print(self):
        print(f"\n{'='*30} {self.__class__.__name__} {'='*30}\n\n{self.content}")


In [16]:
# Creating a sequence of messages with logical flow
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm good, thank you! How can I assist you today?"),
    SecurityProjectManager(content="Can you tell me a joke?"),
    AIMessage(content="Sure! Why don't scientists trust atoms? Because they make up everything!"),
    HumanMessage(content="That's funny! Can you help me with some math?"),
    AIMessage(content="Of course! What math problem do you need help with?"),
    SecurityProjectManager(content="What's the square root of 144?"),
    AIMessage(content="The square root of 144 is 12."),
    ToolMessage(content="Math tool used to calculate the square root of 144.", tool_call_id="12345"),
]

for message in messages:
    message.pretty_print()

================================ System Message ================================

You are a helpful assistant.
================================ Human Message =================================

Hello, how are you?
================================== Ai Message ==================================

I'm good, thank you! How can I assist you today?

============================== SecurityProjectManager ==============================

Can you tell me a joke?
================================== Ai Message ==================================

Sure! Why don't scientists trust atoms? Because they make up everything!
================================ Human Message =================================

That's funny! Can you help me with some math?
================================== Ai Message ==================================

Of course! What math problem do you need help with?

============================== SecurityProjectManager ==============================

What's the square root of 144?
==========

In [4]:
messages[0].type

'system'